In [1]:
import h5py
import numpy as np
import imageio
from cellpose import models
from cellstitch.pipeline import *
from cellstitch.evaluation import *
from cellstitch.utils import *
import pandas as pd

# Pipeline Benchmark

Comparision between cellstitch (2D), cellpose3D (2.5D), plantseg (3D), cellstitch3D; using the same training set.

### PlantSeg
- First, created a plantseg virtual enviroment: 
`conda create -n plant-seg -c pytorch -c conda-forge -c lcerrone -c awolny pytorch=1.9 pytorch-3dunet=1.3.7 plantseg` 
- activate the environment: `conda activate plant-seg` 
- download the ovules test dataset: https://osf.io/uzq3w/ to `../data/ovules/plantseg_test/` 
- set the `path` in `plantseg.yaml` to `../data/ovules/plantseg_test/` 
- perform segmentation with the `confocal_unet_bce_dice_ds1x` by running `plantseg --config plantseg.yaml`
- the predictions are saved to `../data/ovules/plantseg_test/PreProcessing/confocal_unet_bce_dice_ds1x/
`

In [2]:
test_filenames = ["N_294_final_crop_ds2.npy", 
                 "N_435_final_crop_ds2.npy",
                 "N_441_final_crop_ds2.npy",
                 "N_511_final_crop_ds2.npy",
                 "N_522_final_crop_ds2.npy",
                 "N_590_final_crop_ds2.npy",
                 "N_593_final_crop_ds2.npy"]

In [ ]:
plantseg_results_folder = "../data/ovules/plantseg_test/PreProcessing/confocal_unet_bce_dice_ds1x/MultiCut/"

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename) 
    
    with h5py.File("%s/%s_predictions_multicut.h5" % (plantseg_results_folder, test_filename), "r") as f:
        plantseg = np.array(list(f['segmentation'])) 
        
    plantseg[np.where(plantseg == 1)] = 0 # plantseg use 1 as labels
    np.save("./results/ovules/pipeline/plantseg/%s.npy" % test_filename, plantseg)

### Train cellpose model from scratch
First, need to prepare training data for cellpose.

In [2]:
train_filenames = ["N_404_ds2x.npy", 
                  "N_405_A_ds2x.npy", 
                  "N_405_B_ds2x.npy", 
                  "N_416_ds2x.npy",
                  "N_422_ds2x.npy",
                  "N_425_ds2x.npy",
                  "N_428_ds2x.npy",
                  "N_440_ds2x.npy",
                  "N_445_ds2x.npy",
                  "N_449_ds2x.npy",
                  "N_450_ds2x.npy", 
                  "N_451_ds2x.npy",
                  "N_454_ds2x.npy",
                  "N_457_ds2x.npy",
                  "N_458_ds2x.npy",
                  "N_487_ds2x.npy",
                  "N_509_ds2x.npy",
                  "N_512_ds2x.npy",
                   "N_517_ds2x.npy",
                  "N_534_ds2x.npy",
                  "N_535_ds2x.npy",
                  "N_536_ds2x.npy"]

ovules_folder = "../DATA/ovules"
cellpose_folder = "../DATA/ovules/cellpose_train"

In [3]:
for train_filename in train_filenames: 
    img = np.load("%s/raw/%s" % (ovules_folder, train_filename))
    labels = np.load("%s/labels/%s" % (ovules_folder, train_filename)) 
    depth = img.shape[0] 
    
    for i in range(depth): 
        imageio.imwrite("%s/%s_%s.tif" % (cellpose_folder, train_filename, i), img[i])
        imageio.imwrite("%s/%s_%s_masks.tif" % (cellpose_folder, train_filename, i), labels[i])

`python -m cellpose --train --dir ../DATA/ovules/cellpose_train --pretrained_model None --n_epochs 100  --verbose` 

### Generate cellpose3d results

In [4]:
model_dir = '../DATA/ovules/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2023_05_08_09_31_06.231473'

In [5]:
ovules_folder = "../DATA/ovules"

In [6]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [7]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("%s/raw/%s" % (ovules_folder, test_filename)) 
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/ovules/pipeline/cellpose3d/%s" % test_filename, masks) 

Starting N_294_final_crop_ds2.npy
Starting N_435_final_crop_ds2.npy
Starting N_441_final_crop_ds2.npy
Starting N_511_final_crop_ds2.npy
Starting N_522_final_crop_ds2.npy
Starting N_590_final_crop_ds2.npy
Starting N_593_final_crop_ds2.npy


### Generate cellstitch results

In [7]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("%s/raw/%s" % (ovules_folder, test_filename)) 
    
    cellstitch, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0])
    cellstitch = np.array(cellstitch)

    yz_masks, _, _ = model.eval(list(img.transpose(1,0,2)), flow_threshold=flow_threshold, channels = [0,0])
    yz_masks = np.array(yz_masks).transpose(1,0,2)

    xz_masks, _, _ = model.eval(list(img.transpose(2,1,0)), flow_threshold=flow_threshold, channels = [0,0])
    xz_masks = np.array(xz_masks).transpose(2,1,0)

    full_stitch(cellstitch, yz_masks, xz_masks)
    
    np.save("./results/ovules/pipeline/cellstitch/%s" % test_filename, cellstitch) 

Starting N_294_final_crop_ds2.npy
Starting N_435_final_crop_ds2.npy


Starting N_441_final_crop_ds2.npy
Starting N_511_final_crop_ds2.npy


Starting N_522_final_crop_ds2.npy


Starting N_590_final_crop_ds2.npy


Starting N_593_final_crop_ds2.npy


# Ovules with High Anistropy 

### PlantSeg
- activate the environment: `conda activate plant-seg` 
- make sure to set the `path` in plantseg.yaml to `../data/ovules_subsampled/plantseg_test/`
- perform segmentation with the `confocal_unet_bce_dice_ds1x` by running `plantseg --config plantseg.yaml`

In [ ]:
plantseg_results_folder = "../data/ovules_subsampled/plantseg_test/PreProcessing/confocal_unet_bce_dice_ds1x/MultiCut/"

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename) 
    
    with h5py.File("%s/%s_predictions_multicut.h5" % (plantseg_results_folder, test_filename), "r") as f:
        plantseg = np.array(list(f['segmentation'])) 
        
    plantseg[np.where(plantseg == 1)] = 0 # plantseg use 1 as labels
    np.save("./results/ovules_subsampled/pipeline/plantseg/%s.npy" % test_filename, plantseg)

### Generate cellpose3d results

In [ ]:
model_dir = '../data/ovules/cellpose_train/models/cellpose_residual_on_style_on_concatenation_off_cellpose_train_2022_09_19_14_23_42.578556'

In [ ]:
subsampled_ovules_folder = "../data/ovules_subsampled/"

In [ ]:
flow_threshold = 1
model = models.CellposeModel(gpu=True, pretrained_model=model_dir)

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("%s/raw/%s.npy" % (subsampled_ovules_folder, test_filename)) 
    masks, _, _ = model.eval(img, do_3D=True, flow_threshold=flow_threshold, channels = [0,0]) 
    np.save("./results/ovules_subsampled/pipeline/cellpose3d/%s.npy" % test_filename, masks) 

### Generate cellstitch results

In [ ]:
for test_filename in test_filenames: 
    print("Starting %s" % test_filename)
    img = np.load("%s/raw/%s.npy" % (subsampled_ovules_folder, test_filename)) 
    
    masks, _, _ = model.eval(list(img), flow_threshold=flow_threshold, channels = [0,0]) 
    cellstitch = np.array(masks)
    full_stitch(cellstitch) 
    np.save("./results/ovules_subsampled/pipeline/cellstitch/%s.npy" % test_filename, cellstitch) 

# Benchmark Results

In [10]:
dataset = 'ovules' # ovules or ovules_subsampled

ap_threshold = 0.5

In [11]:
method = "cellstitch" 

data = [] 
for filename in test_filenames:
    print("Starting %s" % filename)
    labels = np.load('../DATA/%s/labels/%s' % (dataset, filename)) 
    true_num_cells = np.unique(labels).size - 1 
    true_avg_vol = get_avg_vol(labels) 
    
    masks = np.load("./results/%s/pipeline/%s/%s" % (dataset, method, filename))
    
    num_cells = np.unique(masks).size - 1
    d_num_cells = abs(num_cells - true_num_cells) / true_num_cells

    avg_vol = get_avg_vol(masks)
    d_avg_vol = abs(true_avg_vol - avg_vol) / true_avg_vol

    ap, tp, fp, fn = average_precision(labels, masks, ap_threshold)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    
    row = [ 
        filename, 
        d_num_cells, 
        d_avg_vol, 
        ap, 
        tp, 
        fp, 
        fn, 
        precision,
        recall, 
        true_num_cells, 
        true_avg_vol, 
        num_cells, 
        avg_vol
    ]
    
    data.append(row)

df = pd.DataFrame(data, columns=[
    "filename",  
    "d_num_cells", 
    "d_avg_vol", 
    "ap", 
    "tp", 
    "fp", 
    "fn",
    "precision",
    "recall",
    "true_num_cells", 
    "true_avg_vol", 
    "num_cells", 
    "avg_vol"
])

df.to_csv("./results/%s/pipeline/%s.csv" % (dataset, method), index=False)

Starting N_294_final_crop_ds2.npy


/home/yining/anaconda3/envs/segmentation/lib/python3.8/site-packages/cellpose/metrics.py:205: RuntimeWarning: invalid value encountered in true_divide
  iou = overlap / (n_pixels_pred + n_pixels_true - overlap)


Starting N_435_final_crop_ds2.npy
Starting N_441_final_crop_ds2.npy
Starting N_511_final_crop_ds2.npy
Starting N_522_final_crop_ds2.npy
Starting N_590_final_crop_ds2.npy
Starting N_593_final_crop_ds2.npy


### Analyze Results

In [12]:
method = 'cellstitch'

In [13]:
df = pd.read_csv('results/%s/pipeline/%s.csv' % (dataset, method))

In [14]:
df.mean()

/tmp/ipykernel_559304/3698961737.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.mean()


d_num_cells           0.212236
d_avg_vol             0.126543
ap                    0.476312
tp                 1374.285714
fp                  766.857143
fn                  800.428571
precision             0.642519
recall                0.644789
true_num_cells     1487.571429
true_avg_vol      41812.575714
num_cells          1454.000000
avg_vol           37439.755426
dtype: float64

In [15]:
df.std()

/tmp/ipykernel_559304/3390915376.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df.std()


d_num_cells           0.154432
d_avg_vol             0.076523
ap                    0.107317
tp                  654.703422
fp                  384.396683
fn                  613.369616
precision             0.079772
recall                0.135809
true_num_cells      883.144922
true_avg_vol      16328.584315
num_cells           675.544965
avg_vol           10953.323258
dtype: float64